In [13]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the variables
langchain_tracing = os.getenv("LANGCHAIN_TRACING_V2")
langchain_project = os.getenv("LANGCHAIN_PROJECT")
langsmith_api_key = os.getenv("LANGSMITH_API_KEY")


In [14]:
# The import statement will vary depending on your LLM and vector database. This is an example for OpenAI + ChromaDB

from vanna.openai.openai_chat import OpenAI_Chat
from vanna.chromadb.chromadb_vector import ChromaDB_VectorStore

class MyVanna(ChromaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        OpenAI_Chat.__init__(self, config=config)

vn = MyVanna(config={'api_key': langsmith_api_key, 'model': 'gpt-4o-mini'})

# See the documentation for other options


In [3]:
vn

In [4]:
vn.train(ddl="""CREATE TABLE machines (
    machine_id INTEGER PRIMARY KEY,
    machine_name TEXT NOT NULL,
    location TEXT NOT NULL,
    installation_date DATE NOT NULL,
    max_capacity_per_hour INTEGER NOT NULL,
    maintenance_interval_days INTEGER NOT NULL,
    last_maintenance_date DATE,
    status TEXT NOT NULL
);""")

Adding ddl: CREATE TABLE machines (
    machine_id INTEGER PRIMARY KEY,
    machine_name TEXT NOT NULL,
    location TEXT NOT NULL,
    installation_date DATE NOT NULL,
    max_capacity_per_hour INTEGER NOT NULL,
    maintenance_interval_days INTEGER NOT NULL,
    last_maintenance_date DATE,
    status TEXT NOT NULL
);


'ad7add6f-ba14-5719-a3a5-4c27c35cb799-ddl'

In [5]:
vn.train(ddl="""CREATE TABLE shifts (
    shift_id INTEGER PRIMARY KEY,
    shift_name TEXT NOT NULL,
    start_time TEXT NOT NULL,
    end_time TEXT NOT NULL,
    supervisor_id TEXT NOT NULL,
    target_output INTEGER NOT NULL,
    shift_type TEXT NOT NULL,
    active BOOLEAN NOT NULL
);""")

Adding ddl: CREATE TABLE shifts (
    shift_id INTEGER PRIMARY KEY,
    shift_name TEXT NOT NULL,
    start_time TEXT NOT NULL,
    end_time TEXT NOT NULL,
    supervisor_id TEXT NOT NULL,
    target_output INTEGER NOT NULL,
    shift_type TEXT NOT NULL,
    active BOOLEAN NOT NULL
);


'64e24ccf-760a-5746-9d81-a5c75fc3886a-ddl'

In [6]:
vn.train(ddl="""CREATE TABLE employees (
    employee_id TEXT PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    hire_date DATE NOT NULL,
    shift_preference TEXT NOT NULL,
    skill_level TEXT NOT NULL,
    hourly_rate REAL NOT NULL,
    certification TEXT NOT NULL
);""")

Adding ddl: CREATE TABLE employees (
    employee_id TEXT PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    hire_date DATE NOT NULL,
    shift_preference TEXT NOT NULL,
    skill_level TEXT NOT NULL,
    hourly_rate REAL NOT NULL,
    certification TEXT NOT NULL
);


'54318c50-e450-5745-8f5c-20bc54e5e583-ddl'

In [7]:
vn.train(ddl="""CREATE TABLE production_logs (
    production_id INTEGER PRIMARY KEY AUTOINCREMENT,
    date DATE NOT NULL,
    shift_id INTEGER NOT NULL,
    machine_id INTEGER NOT NULL,
    good_units INTEGER NOT NULL,
    scrap_units INTEGER NOT NULL,
    downtime_minutes INTEGER NOT NULL,
    FOREIGN KEY (shift_id) REFERENCES shifts(shift_id),
    FOREIGN KEY (machine_id) REFERENCES machines(machine_id)
);
);""")

Adding ddl: CREATE TABLE production_logs (
    production_id INTEGER PRIMARY KEY AUTOINCREMENT,
    date DATE NOT NULL,
    shift_id INTEGER NOT NULL,
    machine_id INTEGER NOT NULL,
    good_units INTEGER NOT NULL,
    scrap_units INTEGER NOT NULL,
    downtime_minutes INTEGER NOT NULL,
    FOREIGN KEY (shift_id) REFERENCES shifts(shift_id),
    FOREIGN KEY (machine_id) REFERENCES machines(machine_id)
);
);


'b4468de5-2909-56f6-98d7-7edd865e2ebd-ddl'

In [8]:
vn.train(ddl="""CREATE TABLE employee_shifts (
    assignment_id INTEGER PRIMARY KEY AUTOINCREMENT,
    date DATE NOT NULL,
    shift_id INTEGER NOT NULL,
    employee_id TEXT NOT NULL,
    hours_worked REAL NOT NULL,
    overtime_hours REAL NOT NULL,
    attendance_status TEXT NOT NULL,
    FOREIGN KEY (shift_id) REFERENCES shifts(shift_id),
    FOREIGN KEY (employee_id) REFERENCES employees(employee_id)
);
);""")

Adding ddl: CREATE TABLE employee_shifts (
    assignment_id INTEGER PRIMARY KEY AUTOINCREMENT,
    date DATE NOT NULL,
    shift_id INTEGER NOT NULL,
    employee_id TEXT NOT NULL,
    hours_worked REAL NOT NULL,
    overtime_hours REAL NOT NULL,
    attendance_status TEXT NOT NULL,
    FOREIGN KEY (shift_id) REFERENCES shifts(shift_id),
    FOREIGN KEY (employee_id) REFERENCES employees(employee_id)
);
);


'855e670d-2467-52be-8293-8c3e8fcfbef0-ddl'

In [9]:
import json

# Assuming vn is imported and has a train method
# If vn is part of a specific library, import it accordingly (e.g., from some_library import vn)
# For demonstration, I'll define a dummy vn.train function
def vn_train(question, sql):
    # print(f"Training with: question='{question}', sql='{sql}'")
    vn.train(question=question, sql=sql)

# JSON string
json_data = '''
{
  "training_data": [
    {
      "question": "Which machines had the highest total downtime in 2024?",
      "sql": "SELECT m.machine_name, SUM(pl.downtime_minutes) as total_downtime FROM production_logs pl JOIN machines m ON pl.machine_id = m.machine_id WHERE pl.date LIKE '2024%' GROUP BY m.machine_name ORDER BY total_downtime DESC LIMIT 5"
    },
    {
      "question": "What is the total good units produced per month?",
      "sql": "SELECT strftime('%Y-%m', pl.date) as month, SUM(pl.good_units) as total_good_units FROM production_logs pl GROUP BY strftime('%Y-%m', pl.date) ORDER BY month"
    },
    {
      "question": "How many employees worked overtime in March 2024?",
      "sql": "SELECT COUNT(DISTINCT es.employee_id) as overtime_employees FROM employee_shifts es WHERE es.date LIKE '2024-03%' AND es.overtime_hours > 0"
    },
    {
      "question": "Which shift had the most scrap units produced?",
      "sql": "SELECT s.shift_name, SUM(pl.scrap_units) as total_scrap_units FROM production_logs pl JOIN shifts s ON pl.shift_id = s.shift_id GROUP BY s.shift_name ORDER BY total_scrap_units DESC LIMIT 1"
    },
    {
      "question": "List employees who were absent more than 5 times in 2024.",
      "sql": "SELECT e.employee_id, e.first_name, e.last_name, COUNT(*) as absence_count FROM employee_shifts es JOIN employees e ON es.employee_id = e.employee_id WHERE es.attendance_status = 'Absent' AND es.date LIKE '2024%' GROUP BY e.employee_id, e.first_name, e.last_name HAVING COUNT(*) > 5 ORDER BY absence_count DESC"
    },
    {
      "question": "What is the average downtime per machine location?",
      "sql": "SELECT m.location, AVG(pl.downtime_minutes) as avg_downtime FROM production_logs pl JOIN machines m ON pl.machine_id = m.machine_id GROUP BY m.location ORDER BY avg_downtime DESC"
    },
    {
      "question": "Which supervisor oversaw the most shifts in June 2024?",
      "sql": "SELECT s.supervisor_id, e.first_name, e.last_name, COUNT(DISTINCT es.date || es.shift_id) as shift_count FROM employee_shifts es JOIN shifts s ON es.shift_id = s.shift_id JOIN employees e ON s.supervisor_id = e.employee_id WHERE es.date LIKE '2024-06%' GROUP BY s.supervisor_id, e.first_name, e.last_name ORDER BY shift_count DESC LIMIT 1"
    },
    {
      "question": "How many days had full downtime for all machines in 2024?",
      "sql": "SELECT COUNT(DISTINCT pl.date) as full_downtime_days FROM production_logs pl WHERE pl.downtime_minutes = 480 GROUP BY pl.date HAVING COUNT(*) = (SELECT COUNT(*) * 3 FROM machines)"
    },
    {
      "question": "What is the total hours worked by employees with 'Expert' skill level?",
      "sql": "SELECT SUM(es.hours_worked) as total_hours FROM employee_shifts es JOIN employees e ON es.employee_id = e.employee_id WHERE e.skill_level = 'Expert'"
    },
    {
      "question": "Which machine produced the least good units on Night shifts?",
      "sql": "SELECT m.machine_name, SUM(pl.good_units) as total_good_units FROM production_logs pl JOIN machines m ON pl.machine_id = m.machine_id JOIN shifts s ON pl.shift_id = s.shift_id WHERE s.shift_name = 'Night' GROUP BY m.machine_name ORDER BY total_good_units ASC LIMIT 1"
    }
  ]
}
'''

# Parse JSON
data = json.loads(json_data)

# Iterate and train
for entry in data["training_data"]:
    vn_train(question=entry["question"], sql=entry["sql"])

In [10]:
vn.train(question="Which machines had the highest total downtime in 2024?", sql="SELECT m.machine_name, SUM(pl.downtime_minutes) as total_downtime FROM production_logs pl JOIN machines m ON pl.machine_id = m.machine_id WHERE pl.date LIKE '2024%' GROUP BY m.machine_name ORDER BY total_downtime DESC LIMIT 5")

Insert of existing embedding ID: 395e0306-d1ea-5e21-aac2-74c7ff93c91e-sql
Add of existing embedding ID: 395e0306-d1ea-5e21-aac2-74c7ff93c91e-sql


'395e0306-d1ea-5e21-aac2-74c7ff93c91e-sql'

In [11]:
vn.connect_to_sqlite('syskron_production.db')

In [12]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn)
app.run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on
